In [12]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from tqdm import tqdm
import time, timeit

from procedure_notebook import Procedure
import measurables

In [13]:
proc = Procedure()
proc.setParameters(L = 3)
N_cycles = 30
setup_state_for_next_cycle = Procedure.pass_density_matrix_with_errors
using_state_vectors = False
using_density_matrices = True

In [14]:
#Operator za prekrivanje z osnovnimi stanji
eigenenergies, eigenstates = measurables.tfim_hamiltonian(proc.L, proc.tfim_parameters).eigenstates()


ground_energy = eigenenergies[0]
ground_states = []
for i in range(len(eigenenergies)):
    if np.round(eigenenergies[i], 8) == np.round(ground_energy, 8):
        eigenstates[i] = eigenstates[i].ptrace([i for i in range(proc.L)])
        ground_states.append(eigenstates[i] * eigenstates[i].dag())
    else:
        break 


def measure(state, proc):

    energy = qt.expect(measurables.tfim_hamiltonian(proc.L, proc.tfim_parameters), state)
    state = state.ptrace([i for i in range(proc.L)])
    
    product = 0
    for gstate in ground_states:
        product += Procedure.abs_squared_scalarproduct(state, gstate)

    return [energy, product]




In [ ]:
bigdata = []

errors = [0, 0.001, 0.01, 0.1, 0.2]

for error in errors:
    print(f"start {error}")
    proc.error_percentage = error
    data = proc.runProcedure(N_cycles=50, measure=measure, setup_state_for_next_cycle=Procedure.pass_density_matrix_with_errors, using_density_matrices=True)
    bigdata.append(data)
    print(f"done {error}")


In [ ]:
fig, axs = plt.subplots(2,1)
ax1, ax2 = axs

for i in range(len(errors)):
    data = bigdata[i]
    energies = []
    products = []

    #for i in range(len(data[0,:,0])):
    #    for j in range(len(data[0,0,:])):
    #        energies.append(data[0,i,j])
    #        products.append(data[1,i,j])

    energies = data[0,:,-1]
    products = data[1,:,-1]

    ax1.plot(energies, label = f"{100 * error:0.0f}% error")
    ax2.plot(products, label = f"{100 * error:0.0f}% error")
